In [1]:
import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def extract_df(path):
    images= []
    labels= []
    
    for folder in os.listdir(path):
        folderpath= os.path.join(path, folder)
        # 跳过 .ipynb_checkpoints 文件夹
        if folder == ".ipynb_checkpoints":
            continue
            
        for image in os.listdir(folderpath):
            images.append(os.path.join(folderpath, image))
            labels.append(folder)

    return pd.DataFrame({
        "images": images,
        "labels": labels,
    })

In [4]:
df_trn= extract_df('./Training_Data_Sample')

df_vld= extract_df('./Training_Data_Sample')


df_tst= extract_df('./Training_Data_Sample')

# 创建标签到索引的映射
unique_labels = df_trn['labels'].unique()
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
index_to_label = {idx: label for label, idx in label_to_index.items()}

# 确认映射正确
print("Label to Index Mapping:", label_to_index)

Label to Index Mapping: {'Beetle': 0, 'Butterfly': 1, 'Cat': 2, 'Cow': 3, 'Dog': 4, 'Elephant': 5, 'Gorilla': 6, 'Hippo': 7, 'Lizard': 8, 'Monkey': 9, 'Mouse': 10, 'Panda': 11, 'Spider': 12, 'Tiger': 13, 'Zebra': 14}


In [5]:
# n_labels= len(df_trn['labels'].unique())
# n_labels
# IMAGE_TRGT_SIZE= (256, 256)

In [6]:
class AnimalDataset(Dataset):
    def __init__(self, dataframe, transform=None, label_to_index=None):
        self.dataframe = dataframe
        self.transform = transform
        self.label_to_index = label_to_index

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.label_to_index[label]  # 映射标签为数值
        return image, label

In [7]:
# 数据变换与加载
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

train_dataset = AnimalDataset(df_trn, transform=transform, label_to_index=label_to_index)
val_dataset = AnimalDataset(df_vld, transform=transform, label_to_index=label_to_index)
test_dataset = AnimalDataset(df_tst, transform=transform, label_to_index=label_to_index)

# train_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=24, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)
# cpu带不动改成3了。
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=3, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=False)

In [8]:
# conv_model = nn.Sequential(
#     nn.Conv2d(3, 1024, kernel_size=3),
#     nn.ELU(),
#     nn.MaxPool2d(kernel_size=2),

#     nn.Conv2d(1024, 512, kernel_size=5),
#     nn.ELU(),
#     nn.MaxPool2d(kernel_size=2),

#     nn.Conv2d(512, 256, kernel_size=3),
#     nn.ELU(),
#     nn.MaxPool2d(kernel_size=2),

#     nn.Conv2d(256, 128, kernel_size=3),
#     nn.ELU(),
#     nn.MaxPool2d(kernel_size=2),

#     nn.Flatten()
# )


# sample_input = torch.randn(1, 3, 256, 256) 
# output = conv_model(sample_input) 
# flatten_size = output.shape[1] 
# print("Flatten size:", flatten_size)


# model = nn.Sequential(
#     conv_model,
#     nn.Linear(flatten_size, 256),  
#     nn.ELU(),
#     nn.Linear(256, 128),
#     nn.ELU(),
#     nn.Linear(128, len(label_to_index))   
# )

conv_model = nn.Sequential(
    nn.Conv2d(3, 1024, kernel_size=3),
    nn.ELU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Conv2d(1024, 512, kernel_size=5),  
    nn.ELU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Conv2d(512, 256, kernel_size=3),  
    nn.ELU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Conv2d(256, 128, kernel_size=3),  
    nn.ELU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Flatten()  
)

# 计算 Flatten 的输出尺寸
sample_input = torch.randn(1, 3, 256, 256)  
output = conv_model(sample_input)  
flatten_size = output.shape[1]  
print("Flatten size:", flatten_size)

# 全连接层
model = nn.Sequential(
    conv_model,
    nn.Linear(flatten_size, 256),  
    nn.ELU(),
    nn.Linear(256, 128), 
    nn.ELU(),
    nn.Linear(128, 64), 
    nn.ELU(),
    nn.Linear(64, 32), 
    nn.ELU(),
    nn.Linear(32, len(label_to_index)),  
    nn.Softmax(dim=1)  
)


Flatten size: 21632


In [9]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(16):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            val_acc += (outputs.argmax(1) == labels).sum().item()

    print(f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Acc: {val_acc / len(val_dataset)}")



Epoch 1, Val Loss: 76.53567636013031, Val Acc: 0.08333333333333333


KeyboardInterrupt: 

In [ ]:
model.eval()
test_acc = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        test_acc += (outputs.argmax(1) == labels).sum().item()

print(f"Test Accuracy: {test_acc / len(test_dataset)}")